In [67]:
%load_ext lab_black
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import re
import numpy as np
import math
import collections

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [68]:
input = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""

with open("../inputs/d15.txt", "r") as input_file:
    test_input = input_file.read()

In [69]:
arr = re.findall("\d", test_input)
risk_mat = np.array(list(map(int, arr))).reshape(-1, int(math.sqrt(len(arr))))
risk_mat.shape

(100, 100)

In [70]:
from collections import defaultdict


class Graph:
    def __init__(self):
        """
        self.edges is a dict of all possible next nodes
        e.g. {'X': ['A', 'B', 'C', 'E'], ...}
        self.weights has all the weights between two nodes,
        with the two nodes as a tuple as the key
        e.g. {('X', 'A'): 7, ('X', 'B'): 2, ...}
        """
        self.edges = defaultdict(list)
        self.weights = {}

    def add_edge(self, from_node, to_node, weight):
        # Note: assumes edges are bi-directional
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight

In [71]:
graph = Graph()

In [72]:
def get_adjacent(i):
    x, y = i
    adj = np.array(
        [
            (x + 1, y),
            (x, y + 1),
        ]
    )
    adj = adj[np.apply_along_axis(all, 1, (adj >= 0) & (adj < risk_mat.shape[0]))]
    return list(map(tuple, adj))

In [73]:
edges = []
for i_row in range(risk_mat.shape[0]):
    for i_col in range(risk_mat.shape[1]):
        i = (i_row, i_col)
        i_adjacent = get_adjacent(i)
        for adjacent in i_adjacent:
            edges.append((i, adjacent, risk_mat[adjacent]))

In [74]:
for edge in edges:
    graph.add_edge(*edge)

In [75]:
def dijsktra(graph, initial, end):
    # shortest paths is a dict of nodes
    # whose value is a tuple of (previous node, weight)
    shortest_paths = {initial: (None, 0)}
    current_node = initial
    visited = set()

    while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = shortest_paths[current_node][1]

        for next_node in destinations:
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in shortest_paths:
                shortest_paths[next_node] = (current_node, weight)
            else:
                current_shortest_weight = shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    shortest_paths[next_node] = (current_node, weight)

        next_destinations = {
            node: shortest_paths[node] for node in shortest_paths if node not in visited
        }
        if not next_destinations:
            return "Route Not Possible"
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])

    # Work back through destinations in shortest path
    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_paths[current_node][0]
        current_node = next_node
    # Reverse path
    path = path[::-1]
    return path

In [76]:
paths = dijsktra(graph, (0, 0), (risk_mat.shape[0] - 1, risk_mat.shape[0] - 1))

In [77]:
risk_sum = 0
for i_p in range(len(paths) - 1):
    risk_sum += risk_mat[paths[i_p + 1]]
risk_sum

383